In [0]:
!pip3 install fasttext
!pip3 install pandas numpy tensorflow sklearn scipy six keras matplotlib pydot

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2387261 sha256=4923b74567263aacce0a85925f9adc5d0fb14ffe17f2d46b37a124425b12f1bd
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [0]:
!pip3 install jsonlines

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Input, Embedding, LSTM, Dropout
from keras.models import Model, Sequential
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split

from small_dataset import get_small_dataset
# from large_dataset import get_large_dataset

ModuleNotFoundError: ignored

In [0]:
df = get_small_dataset()
#df.describe()

FileNotFoundError: ignored

In [0]:
X = df['title']
y = df['label']

In [0]:
"""
Code courtesy of https://keras.io/examples/pretrained_word_embeddings/
Inspiration also from https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
"""
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.


FileNotFoundError: ignored

In [0]:
# vectorize the text samples (titles) into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length = max(len(sequences[i]) for i in range(len(sequences)))
input_length = max_length + 1

padded_X = pad_sequences(sequences, maxlen=input_length, padding="post")

#cat_y = to_categorical(np.asarray(y))
print('Shape of data tensor:', padded_X.shape)
print('Shape of label tensor:', y.shape)

In [0]:
# split data into training and test set

# percentage of test data
percentage = 0.2

# # split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(padded_X, y, test_size = percentage) #random_state = 51



print("total training examples: %d\n total test examples: %d" % (len(X_train), len(X_test)))

In [0]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=input_length,
                            trainable=False)

In [0]:
# build model
lstm_out = 256
dense_out= 64
dropout = 0.3

sequence_input = Input(shape=(input_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

model = Sequential()
#model.add(sequence_input)
model.add(embedding_layer)
model.add(LSTM(lstm_out, dropout=0.1, recurrent_dropout=0.1))
# model.add(Dense(dense_out, activation='relu'))
# model.add(Dropout(rate=dropout))
model.add(Dense(1, activation='relu'))

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

In [0]:
### Create model checkpoint
output_dir = os.getcwd()
modelcheckpoint = ModelCheckpoint(filepath = output_dir + "/RNNweights.{epoch:02d}-{val_loss:.2f}.hdf5",
                                 monitor='val_loss', verbose=1, save_best_only=True)
callbacks_list = [modelcheckpoint, EarlyStopping(monitor = "val_loss", patience = 5)]

In [0]:
#Hyper-Params
epochs = 50
batch_size = 32


# train model
history = model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs, verbose=1, validation_split = 0.2, callbacks = callbacks_list)

#print validation data
# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss   = history.history['val_loss']
train_acc  = history.history['accuracy']
val_acc    = history.history['val_accuracy']
xc         = range(11)

plt.figure()
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)


In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
print(model.metrics_names)
model.evaluate(X_test, y_test)


In [0]:
model = tf.keras.Sequential()
model.add(layers.Embedding(input_dim=1000, output_dim=64))

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model.add(layers.GRU(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model.add(layers.SimpleRNN(128))

model.add(layers.Dense(10, activation='softmax'))

model.summary()